https://yanlinc.medium.com/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6

In [4]:
from chemdataextractor import Document
import os 


In [6]:
pdf_folder = r'C:\Users\phili\techlabs\nlp_delft\paper'
pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

In [7]:
folder_name = "csv"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'csv' already exists.


In [8]:
csv_dir = r'C:\Users\phili\techlabs\nlp_delft\csv'

Separate every paragraph and save it into dataframe

In [5]:
for pdf_file in os.listdir(pdf_folder):

    f = open(os.path.join(pdf_folder, pdf_file),'rb')
    try:
        doc = Document.from_file(f)
        ltext = []
        for i in range(len(doc.elements)):
            ltext.append(str(doc.elements[i]))
        df = pd.DataFrame(ltext)
        df.to_csv(os.path.join(csv_dir,f'{pdf_file}.csv'))
            
    except:
        pass

Make big dataframe

In [4]:
# Run in terminal or command prompt
# python3 -m spacy download en
import os
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import pyLDAvis
# import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
dataframes = []

for filename in os.listdir(csv_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_dir, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

big_dataframe = pd.concat(dataframes, ignore_index=True)

In [10]:
big_dataframe

,Unnamed: 0,0
0,0,Article\nProcess Development for Methyl Isobut...
1,1,", Raed R. Alkathiri 1 and Abdulaziz A. Bagabas 2"
2,2,"1 Chemical Engineering Department, King Saud U..."
3,3,"Technology (KACST), P.O. Box 6086, Riyadh 1144..."
4,4,* Correspondence: arabiah@ksu.edu.sa; Tel.: +9...
...,...,...
6382,560,"10-2022-055782, 4 May 2022."
6383,561,"267. Xue, Y.; Qiao, X.; Post, M.; Jia, Z.; Li,..."
6384,562,November 2022.
6385,563,"268. Yoshihiko, H.; Katsuhiko, H. Production o..."


In [11]:
big_dataframe.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [12]:
big_dataframe = big_dataframe.rename(columns={"0": "Paragraph"})

In [13]:
big_dataframe = big_dataframe.drop(['Unnamed: 0'], axis=1)

In [14]:
big_dataframe['count'] = big_dataframe['Paragraph'].str.split().str.len()


In [15]:
# Convert to list
data =big_dataframe['Paragraph'].values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['Article Process Development for Methyl Isobutyl Ketone Production Using the '
 'Low-Pressure One-Step Gas-Phase Selective Hydrogenation of Acetone '
 'Abdulrahman A. Al-Rabiah 1,*']


In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['article', 'process', 'development', 'for', 'methyl', 'isobutyl', 'ketone', 'production', 'using', 'the', 'low', 'pressure', 'one', 'step', 'gas', 'phase', 'selective', 'hydrogenation', 'of', 'acetone', 'abdulrahman', 'al', 'rabiah']]


In [17]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [20]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

['article process development methyl isobutyl ketone production use pressure step gas phase hydrogenation acetone', 'rae alkathiri']


In [21]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,

                             stop_words='english',             

                             lowercase=True,                   

                             token_pattern='[a-zA-Z0-9]{3,}',  

                             )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [22]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [24]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method='online', learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=10, n_jobs=-1, perp_tol=0.1,
 random_state=100, topic_word_prior=None,
 total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)

In [25]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -231883.87953138252
Perplexity:  317.60355529563293
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [27]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),n_jobs=1,
       param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_topics': [10, 15, 20, 25, 30]},
             return_train_score='warn')

In [28]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -54147.31432432652
Model Perplexity:  322.37339194436834


In [31]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.010000,0.010000,0.070000,0.070000,0.080000,0.590000,0.010000,0.010000,0.010000,0.150000,5
Doc1,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0
Doc2,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.770000,0.030000,0.030000,0.030000,6
Doc3,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,7
Doc4,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,9
Doc5,0.000000,0.000000,0.000000,0.090000,0.010000,0.450000,0.320000,0.080000,0.000000,0.050000,5
Doc6,0.010000,0.010000,0.010000,0.010000,0.010000,0.360000,0.010000,0.010000,0.010000,0.570000,9
Doc7,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,7
Doc8,0.010000,0.010000,0.010000,0.010000,0.010000,0.100000,0.010000,0.750000,0.100000,0.010000,7
Doc9,0.390000,0.010000,0.010000,0.010000,0.010000,0.240000,0.110000,0.220000,0.010000,0.010000,0


In [33]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names_out()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,absorber,absorption,accept,access,accord,account,acetate,acetone,achieve,acid,...,weight,wind,work,world,write,writing,www,year,yield,zeolite
Topic0,0.100000,0.100000,12.889754,0.100020,0.100010,0.100009,0.100023,0.100008,0.100005,0.100021,...,0.100018,0.100006,0.100009,0.100063,0.100000,0.100004,0.100018,0.100016,0.100027,0.100007
Topic1,0.100000,0.100000,0.100000,0.100009,0.100023,0.100014,0.100006,0.100011,0.100016,0.100012,...,0.100020,0.100005,0.100033,0.100007,20.511680,9.723993,0.100002,0.100017,0.100017,0.100000
Topic2,0.100001,0.100009,0.100000,138.021064,0.100020,0.100009,0.100009,0.100005,0.100007,0.100011,...,0.100005,0.100017,0.100014,0.100006,0.100001,0.100002,15.463960,0.100027,0.100009,0.100002
Topic3,0.100015,0.100006,0.100000,0.100001,3.023076,0.100029,0.100014,0.100020,0.104714,0.100008,...,0.100046,0.100002,14.144068,0.100009,0.100020,0.100001,0.100000,0.100006,0.100027,0.100006
Topic4,11.124359,15.466345,0.100000,0.100003,0.100025,12.358474,0.100012,0.100006,2.122122,0.100006,...,15.736559,0.100006,0.105214,0.100026,0.100009,0.100001,0.100000,0.100012,0.100012,0.100002


In [35]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,chem,ton,market,parameter,research,fraction,table,assumption,beta,flow,alternative,select,rate,catalysis,fund
Topic 1,patent,footprint,datum,ethanol,preparation,pem,study,sub,review,author,batch,publish,resource,methodology,contribution
Topic 2,access,cycle,com,power,life,value,https,article,ciency,exergy,min,combine,impact,condition,mention
Topic 3,stream,heat,reactor,feed,temperature,pressure,figure,stage,recycle,ratio,bar,number,cool,exchanger,rate
Topic 4,gas,phase,formalin,steam,optimization,air,loss,variable,evaluation,separator,generation,turbine,site,heat,metal
Topic 5,process,reaction,use,phase,table,method,figure,catalyst,model,selectivity,result,step,simulation,obtain,equilibrium
Topic 6,cost,process,unit,consumption,utility,energy,capital,electricity,year,use,base,design,calculate,equipment,material
Topic 7,methanol,production,case,use,hydrogen,produce,formaldehyde,solvent,fuel,water,solution,chemical,process,dme,concentration
Topic 8,acid,water,product,column,distillation,butanol,mixture,use,waste,cation,extraction,content,component,encyclopedia,separation
Topic 9,isophorone,acetone,catalyst,condensation,energy,synthesis,vapor,oxide,crossref,carbon,base,analysis,report,self,liquid


In [37]:
Topics = ["no_process","no_process","no_process","no_process","no_process", 
          "process", "process", "process", "process", "no_process"]
df_topic_keywords["Topics"]=Topics
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Topics
Topic 0,chem,ton,market,parameter,research,fraction,table,assumption,beta,flow,alternative,select,rate,catalysis,fund,no_process
Topic 1,patent,footprint,datum,ethanol,preparation,pem,study,sub,review,author,batch,publish,resource,methodology,contribution,no_process
Topic 2,access,cycle,com,power,life,value,https,article,ciency,exergy,min,combine,impact,condition,mention,no_process
Topic 3,stream,heat,reactor,feed,temperature,pressure,figure,stage,recycle,ratio,bar,number,cool,exchanger,rate,no_process
Topic 4,gas,phase,formalin,steam,optimization,air,loss,variable,evaluation,separator,generation,turbine,site,heat,metal,no_process
Topic 5,process,reaction,use,phase,table,method,figure,catalyst,model,selectivity,result,step,simulation,obtain,equilibrium,process
Topic 6,cost,process,unit,consumption,utility,energy,capital,electricity,year,use,base,design,calculate,equipment,material,process
Topic 7,methanol,production,case,use,hydrogen,produce,formaldehyde,solvent,fuel,water,solution,chemical,process,dme,concentration,process
Topic 8,acid,water,product,column,distillation,butanol,mixture,use,waste,cation,extraction,content,component,encyclopedia,separation,process
Topic 9,isophorone,acetone,catalyst,condensation,energy,synthesis,vapor,oxide,crossref,carbon,base,analysis,report,self,liquid,no_process


In [39]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), 1:14].values.tolist()
    
    # Step 5: Infer Topic
    infer_topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), -1]
    
    #topic_guess = df_topic_keywords.iloc[np.argmax(topic_probability_scores), Topics]
    return infer_topic, topic, topic_probability_scores

In [50]:
# Predict the topic
mytext = ["""
In order to improve the membrane performances, the membrane layer can be formed by
different films grown in subsequent hydrothermal steps by following the so called multi
in situ crystallization (MISC) method (Vroon et al., 1998) (see Fig. 18.8). The membranes
produced by this method are, in general, defect-free membranes."""]
infer_topic, topic, prob_scores = predict_topic(text = mytext)
print(topic)
print(infer_topic)

['reaction', 'use', 'phase', 'table', 'method', 'figure', 'catalyst', 'model', 'selectivity', 'result', 'step', 'simulation', 'obtain']
process
